## COVID Pipeline Deployment

### Import libraries

In [56]:
import kfp
import os
import calendar
import time
from kubernetes import client


### Component files Declarations

In [57]:
component_root_datadownload= 'components/covid_download_data/'
component_root_preprocess = 'components/covid_preprocess/'
component_root_train = 'components/covid_train/'
component_root_serve = 'components/covid_serve/'
component_root_upload_result = 'components/covid_upload_result/'

#### Components Description
covid_datadownload_op &nbsp; - &nbsp; Downloads dataset from GCP bucket and stores in nfs-volume. <br>
covid_preprocess_op &nbsp; - &nbsp; Preprocess the raw dataset to make it suitable for training. <br>
covid_train_op &nbsp; - &nbsp; Load dataset from nfs-volume, train COVID-19 model and saves model in nfs-volume. <br>
covid_serve_op &nbsp; - &nbsp; Load COVID-19 model from nfs-volume and serve the model using tf-serving for later prediction. <br> 
covid_upload_result_op &nbsp; - &nbsp; Uploading the prediction results into GCS bucket for the sake of visualisation. <br> 



### Load components from respective .YAML config files

In [58]:
covid_datadownload_op = kfp.components.load_component_from_file(os.path.join(component_root_datadownload, 'component.yaml')) 
covid_preprocess_op = kfp.components.load_component_from_file(os.path.join(component_root_preprocess, 'component.yaml'))
covid_train_op = kfp.components.load_component_from_file(os.path.join(component_root_train, 'component.yaml'))
covid_serve_op = kfp.components.load_component_from_file(os.path.join(component_root_serve, 'component.yaml'))
covid_upload_result_op = kfp.components.load_component_from_file(os.path.join(component_root_upload_result, 'component.yaml'))

### Define Volume and Volume Mounts

In [59]:
nfs_pvc = client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs')
nfs_volume = client.V1Volume(name='nfs', persistent_volume_claim=nfs_pvc)
nfs_volume_mount = client.V1VolumeMount(mount_path='/mnt/', name='nfs')


### Define pipeline function

In [60]:
# Creating timestamp
timestamp = str(calendar.timegm(time.gmtime()))
print(timestamp)

def covid_pipeline():
    
    #Define Task for Downloading Dataset
    covid_datadownload_task = covid_datadownload_op()
    covid_datadownload_task.add_volume(nfs_volume)
    covid_datadownload_task.add_volume_mount(nfs_volume_mount)
    
    
    #Define Task for Preprocess covid data
    covid_preprocess_task = covid_preprocess_op()
    covid_preprocess_task.add_volume(nfs_volume)
    covid_preprocess_task.add_volume_mount(nfs_volume_mount)
    covid_preprocess_task.after(covid_datadownload_task)
    
    #Define Task for Training model for COVID-19 prediction
    covid_train_task = covid_train_op(epochs=10)
    covid_train_task.add_volume(nfs_volume)
    covid_train_task.add_volume_mount(nfs_volume_mount)
    covid_train_task.after(covid_preprocess_task)
    
    #Define Task for Serving for COVID-19 prediction
    covid_serve_task = covid_serve_op(timestamp='20200427')
    covid_serve_task.add_volume(nfs_volume)
    covid_serve_task.add_volume_mount(nfs_volume_mount)
    covid_serve_task.after(covid_train_task)
    
    #Define Task for uploading COVID-19 prediction result in GCS bucket
    covid_upload_result_task = covid_upload_result_op()
    covid_upload_result_task.add_volume(nfs_volume)
    covid_upload_result_task.add_volume_mount(nfs_volume_mount)
    covid_upload_result_task.after(covid_train_task)

    

#Create a pipeline run
kfp.Client().create_run_from_pipeline_func(covid_pipeline, arguments={})

1587984243


RunPipelineResult(run_id=2d1e358a-4a36-481e-af6a-98bf5f553879)